### Format the histlists and add file names
Henry Ang 10/23/17 /global/scratch/groups/dh/aanderson/

Hitlists' directory:<br/>
/global/scratch/groups/dh/aanderson/formatted_bighits.txt<br/>
/global/scratch/groups/dh/aanderson/formatted_smallhits.txt

A Formatted hitlist looks like this:

filename1;GoogleDriveFileId<br/>
1,3,4<br/>

filename2;GoogleDriveFileId<br/>
1,2,3<br/>

In [ ]:
projectDir = "/global/scratch/groups/dh/aanderson/"
resultsDir = projectDir + "results/"
archiveDir = projectDir + "run_archive/"

#### Authentication from main notebook

In [ ]:
import os
import codecs
import httplib2
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from apiclient import discovery, errors
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

In [ ]:
SCOPES = 'https://www.googleapis.com/auth/drive'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'gDriveConnect'

In [ ]:
import argparse
parser = argparse.ArgumentParser(parents=[tools.argparser])
parser.add_argument('-f', help=argparse.SUPPRESS)

flags = parser.parse_known_args()[0]
flags.noauth_local_webserver = True

In [ ]:
def get_credentials():
    
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir, 'gDriveConnect.json')
    
    store = Storage(credential_path)    
    credentials = store.get()
    
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print('Storing credentials to ' + credential_path)
        
    return credentials

In [ ]:
credentials = get_credentials()
print('credential_path:', credentials)
http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', http=http)

In [ ]:
def createFormattedHistlist(filePath, newFilePath):
    try:
        with open(filePath, 'r', encoding='utf-8') as f:
            content = f.readlines()
            f.close()
    except:
        return
    lines = [line.strip() for line in content]
    hitFiles = {}
    for line in lines:
        googleFileID, rest = line[:line.find(';')], line[line.find(';') + 1:]
        pageNumber, rest = rest[:rest.find(';')], rest[rest.find(';') + 1:]
        matchList = rest 
        if googleFileID in hitFiles:
            hitFiles[googleFileID].append((pageNumber, matchList))
        else:
            hitFiles[googleFileID] = [(pageNumber, matchList)]

    for hitFileId in hitFiles:
        try:
            file = service.files().get(fileId=hitFileId).execute()
            name = file['name']
        except Error:
            name = 'unable to get file name'
        
        pageNumbers = [file[0] for file in hitFiles[hitFileId]]
        firstLine = "{0};{1}".format(name, hitFileId) + '\n'
        secondLine = ",".join(pageNumbers) + '\n'
        #print(firstLine)
        #print (secondLine)
        with open(newFilePath, 'a', encoding='utf-8') as f2:
            f2.writelines([firstLine, secondLine, '\n'])
            f2.close()
        
        


In [ ]:
SHL, SHLF = "smallhitlist.txt", projectDir + "formatted_smallhits.txt"
BHL, BHLF = "bighitlist.txt", projectDir + "formatted_bighits.txt"
try:
    os.remove(projectDir + "formatted_smallhits.txt")
    os.remove(projectDir + "formatted_bighits.txt")
except:
    pass
os.chdir(resultsDir)
dirs = os.listdir(resultsDir)
for resultFolder in dirs:
    resultFolderDir = resultsDir + resultFolder + "/"
    print(resultFolderDir)
    files = os.listdir(resultFolderDir)
    createFormattedHistlist(resultFolderDir + SHL, SHLF)
    createFormattedHistlist(resultFolderDir + BHL, BHLF)
print("Completed.")
        
    

In [ ]:
#testing

os.chdir(resultsDir)
dirs = os.listdir(resultsDir)
os.chdir("Results_716-735")
dirs = os.listdir()
createFormattedHistlist(resultsDir+"Results_716-735/smallhitlist.txt", projectDir + "formatted_bighits.txt")

In [ ]:
#how many pdfs with bighits
with open(projectDir + "formatted_bighits.txt", 'r', encoding='utf-8') as f:
    content = f.readlines()
    f.close()
lines = [line.strip() for line in content]
len(lines)/3

In [ ]:
#how many pdfs with smallhits
with open(projectDir + "formatted_smallhits.txt", 'r', encoding='utf-8') as f:
    content = f.readlines()
    f.close()
lines = [line.strip() for line in content]
len(lines)/3